In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

DATASET_COLUMNS=['review','sentiment']
DATASET_ENCODING = "ISO-8859-1"
df = pd.read_csv('/content/drive/MyDrive/Dataset/IMDB Dataset.csv', encoding=DATASET_ENCODING, names=DATASET_COLUMNS, header = None, nrows = 5000)
print("Dataframe shape:", df.shape)
df.head()

Dataframe shape: (5000, 2)


,review,sentiment
0,review,sentiment
1,One of the other reviewers has mentioned that ...,positive
2,A wonderful little production. <br /><br />The...,positive
3,I thought this was a wonderful way to spend ti...,positive
4,Basically there's a family where a little boy ...,negative


In [ ]:
data=df[['review','sentiment']]
data.head()

,review,sentiment
0,review,sentiment
1,One of the other reviewers has mentioned that ...,positive
2,A wonderful little production. <br /><br />The...,positive
3,I thought this was a wonderful way to spend ti...,positive
4,Basically there's a family where a little boy ...,negative


In [ ]:
# data['review'] = data['review'].replace(4,1)
data['sentiment'].value_counts()

negative     2531
positive     2468
sentiment       1
Name: sentiment, dtype: int64

In [ ]:
data_pos = data[data['sentiment'] == "positive"]
# data_pos = data_pos[0:40]
data_pos.head()

,review,sentiment
1,One of the other reviewers has mentioned that ...,positive
2,A wonderful little production. <br /><br />The...,positive
3,I thought this was a wonderful way to spend ti...,positive
5,"Petter Mattei's ""Love in the Time of Money"" is...",positive
6,"Probably my all-time favorite movie, a story o...",positive


In [ ]:
data_neg = data[data['sentiment'] == "negative"]
# data_neg = data_neg[0:40]
data_neg.head()

,review,sentiment
4,Basically there's a family where a little boy ...,negative
8,"This show was an amazing, fresh & innovative i...",negative
9,Encouraged by the positive comments about this...,negative
11,Phil the Alien is one of those quirky films wh...,negative
12,I saw this movie when I was about 12 when it c...,negative


In [ ]:
import numpy as np
data_pos = data_pos.sample(2468)
data_neg = data_neg.sample(2531)
dataset = pd.concat([data_pos, data_neg])
dataset = dataset.iloc[np.random.permutation(len(dataset))]
dataset=dataset.reset_index(drop=True)
dataset.head()

,review,sentiment
0,I watched this movie so that you don't have to...,negative
1,Tainted look at kibbutz life<br /><br />This f...,negative
2,Across the country and especially in the polit...,positive
3,Honestly - this short film sucks. the dummy us...,negative
4,Wagon Master is a very unique film amongst Joh...,positive


In [ ]:
X=list(dataset['review'])
X[:5]

['I watched this movie so that you don\'t have to! I have great respect for Kris Kristofferson, but what was he thinking? He did this for scale? <br /><br />At least the film\'s title practices truth in advertising, since people and objects routinely disappear throughout the film, adding to the confusion. Kristofferson mentions this in his commentary that even he wasn\'t sure if Genevieve Bujold\'s character really existed. This does not bode well for the viewer being able to follow the story!<br /><br />The "making of" feature was far more interesting than the movie itself. It explores the difficulty cobbling together funding for an indie, even as the film is being shot.<br /><br />To it\'s credit, this movie is visually pleasing and doesn\'t in any way look like a movie made with just slightly over 1M. Too bad the money wasn\'t spent on a better project.',
 'Tainted look at kibbutz life<br /><br />This film is less a cultural story about a boy\'s life in a kibbutz, but the deliberate

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["labeled_sentiment"] = le.fit_transform(df["sentiment"])
df
y=list(df["labeled_sentiment"])[1:]

# y=list(dataset['labeled_sentiment'])
y[:5]

[1, 1, 1, 0, 1]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42,stratify=y)
X_train[:5]

['After some further thought about this film, I find it\'s far too easy to dismiss this as the Boy\'s dream. I have actually received some spiritual strength from Northfork.......Angels do exist....we definitely are entertained by Angels....most of the time we aren\'t even aware of it..... At a point of spiritual and emotional turbidity in my life, I personally really needed this film. Yes, as I wrote before, it speaks to so many......can\'t wait to get to Heaven...<br /><br />"Being so sick of all of the FX and Formula stuff, I found this film to be genuine Cinema. All I can say is it touched me in so many ways, that I still am sorting it all out. North Fork is a wonderful film. One that brings the viewer\'s mind out of the gutter and into the heart. The spiritual aspect is so very intriguing to me. Pay attention, as you\'ll need to use the brain and heart God gave you to follow the story. I think it\'s possibly a bit over the heads of some, but I feel those are the individuals it spe

In [ ]:
!pip install transformers==4.6.0


In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
y_train[:5]

[1, 0, 0, 0, 1]

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
    logging_steps=100,
)

In [ ]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)


trainer.train()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_79', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.6960604228670635}

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-0.12973665,  0.00567051],
       [-0.13656467,  0.01338657],
       [-0.13010158,  0.0083384 ],
       ...,
       [-0.12981796,  0.00842399],
       [-0.13041387,  0.00818098],
       [-0.1274275 ,  0.00639599]], dtype=float32), label_ids=array([1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,trainer.predict(test_dataset)[1]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       506
           1       1.00      1.00      1.00       494

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

